# imports

In [1]:
import pickle
from srgnn_datasets import SRGNN_Map_Dataset, Augment_Matrix_Dataset, SRGNN_sampler, Clusters_Matrix_Dataset
from utils import load_model, get_dataset, load_model_gm
import os

from torch.utils.data import DataLoader

import numpy as np
import pandas as pd

from tqdm import tqdm

import torch
from pytorch_lightning import Trainer
from math import ceil

/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.set_float32_matmul_precision('medium')

In [3]:
def get_metrics_by_hand(model, dataloaders):
    m=[]
    for i, d in enumerate(dataloaders):
        hit,mrr=[],[]
        for batch in tqdm(d, total=ceil(normal_test_dataset.length/opt.batchSize)):
            batch=[x.to('cuda') for x in batch]
            sub_scores, targets=model.predict_step(batch)
            targets=targets.flatten()
            for score, target in zip(sub_scores, targets):
                correct_pred=torch.isin(target - 1, score)
                hit.append(correct_pred.cpu().numpy())
                if not correct_pred:
                    mrr.append(0)
                else:
                    mrr.append(1 / (torch.where(score == target - 1)[0][0] + 1).cpu().numpy())
        m.append((i, 100*np.average(hit),100*np.average(mrr)))
    return m


# otto-recsys

In [4]:
runs=['7dt7uu2i', '31mccq76', 'f42h3b7i', 'z5gthlar', 'icirdid1',
    'q5ncdos3',
 '20x2q8ap',
 'edk5gwaj',
 'eyipcpfr',
 'u1769vwh',
 '7toa2ybx',
 'zrkfqp80',
 '0dqi0cx4',
 'leop64a1',
 'c4l0cw03',
 'rufekv8o',
 'zja7utqy',
 'lc8hf28y',
 'pi1q0ni6',
 '78wwq4e4',

'7oziz2pv',
'77r7ocil',
'umy0p2nx',
'ch778pru',
'dn0mhesi',
'qpljbobb',
's8wy4wxs',
        ]


In [5]:
run_id=runs[0]

model,opt=load_model(run_id)
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

{'dataset': 'otto-recsys', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
data masking start


100%|██████████| 64/64 [00:00<00:00, 307.87it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


done masking


/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [6]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue

    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

{'dataset': 'otto-recsys', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
7dt7uu2i


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 627/627 [00:18<00:00, 34.71it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            31.453794479370117
        test_loss            7.820014953613281
        test_mrr             20.4002685546875
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_cluster

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


31mccq76
Testing DataLoader 0: 100%|██████████| 627/627 [00:17<00:00, 36.72it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            31.452913284301758
        test_loss            7.821662902832031
        test_mrr             20.58791732788086
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augme

...

dn0mhesi
Testing DataLoader 0: 100%|██████████| 627/627 [00:16<00:00, 36.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             32.00474548339844
        test_loss            7.939952373504639
        test_mrr            20.955114364624023
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augme

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


qpljbobb
Testing DataLoader 0: 100%|██████████| 627/627 [00:16<00:00, 37.31it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            31.343982696533203
        test_loss            7.809231281280518
        test_mrr            20.283445358276367
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augme

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


s8wy4wxs
Testing DataLoader 0: 100%|██████████| 627/627 [00:16<00:00, 37.16it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             31.42213249206543
        test_loss            7.875438213348389
        test_mrr            20.393234252929688
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [7]:
result_df=pd.DataFrame(res_df)

In [8]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_noise_p','augment_alg']].groupby(['augment_matrix', 'augment_noise_p', 'augment_alg']).mean()

test_loss   test_hit   test_mrr
augment_matrix augment_noise_p augment_alg                                 
False          0.0             gmm           7.810835  31.417475  20.491862
               0.5             gmm           7.858672  31.493632  20.538345
True           0.5             gmm           7.819558  32.020105  21.179639
                               kmeans        7.823207  32.054985  21.340188
                               raw           7.837602  32.048814  21.161132

# yoochoose 1/64

## baseline models

In [2]:
run_id='7zi9x9w8'

model,opt=load_model(run_id)
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))

{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': None, 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}


In [12]:
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )

data masking start


100%|██████████| 64/64 [00:00<00:00, 271.66it/s]

done masking


In [13]:
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [9]:
runs=[
    '7zi9x9w8'
]

In [10]:
for run_id in runs:
    model,opt=load_model(run_id)
    print('Metrics on normal Adjacency matrix')
    print(run_id)
    trainer.test(model, normal_test_dataloader)

{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': None, 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
Metrics on normal Adjacency matrix
7zi9x9w8


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 559/559 [00:17<00:00, 32.33it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             69.87379455566406
        test_loss           4.3695454597473145
        test_mrr             30.65968894958496
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## augmented models

In [12]:
runs=[ # yoochoose1/64
    'qgkxyze7',
    'r1xr7g4v',
    'itxri54t',
    '7vvqd0ib',
    'jefoas5f',
    'ai6ytfw2',
    'xm3z645m',
    'zf87zj40',    
]

In [13]:
res_df=[]
for run_id in runs:
    model,opt=load_model(run_id)
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    print('Distnace Augmentation:', opt.augment_matrix,
        'Clusters:', opt.augment_clusters,
          'Categories:', opt.augment_categories,
          'Noise std: ', opt.augment_std,
          'base model', opt.augment_old_run_id,
          )
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    metrics['augment_nogmm']=opt.augment_nogmm
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_old_run_id': '7zi9x9w8', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 128, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
qgkxyze7
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8
Testing DataLoader 0: 100%|██████████| 559/559 [00:17<00:00, 32.80it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric        

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


r1xr7g4v
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8
Testing DataLoader 0: 100%|██████████| 559/559 [00:16<00:00, 33.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.14924621582031
        test_loss            4.378706455230713
        test_mrr             30.86641502380371
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

ai6ytfw2
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8
Testing DataLoader 0: 100%|██████████| 559/559 [00:17<00:00, 32.82it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.11174774169922
        test_loss            4.379937171936035
        test_mrr            30.824819564819336
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


xm3z645m
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8
Testing DataLoader 0: 100%|██████████| 559/559 [00:16<00:00, 33.20it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             69.93103790283203
        test_loss            4.372420787811279
        test_mrr            30.606473922729492
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


zf87zj40
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8
Testing DataLoader 0: 100%|██████████| 559/559 [00:16<00:00, 32.94it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.2243423461914
        test_loss            4.360525131225586
        test_mrr              30.74462890625
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [18]:
res_df=pd.DataFrame(res_df)
res_df.drop(columns='run_id').groupby('augment_nogmm').mean()

,test_loss,test_hit,test_mrr
augment_nogmm,,,
2,4.366473,70.077690,30.675551
8,4.377462,70.077744,30.764617
32,4.375669,70.053593,30.743398
128,4.371604,70.033863,30.772902


In [19]:
results={}
for run_id in runs:
    model,opt=load_model(run_id)
    #dataset=get_dataset(opt)
   # am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
     #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
      #                   )
    print(run_id)
    print('Distnace Augmentation:', opt.augment_matrix,
        'Clusters:', opt.augment_clusters,
          'Categories:', opt.augment_categories,
          'Noise std: ', opt.augment_std,
          'base model', opt.augment_old_run_id,
          )
    results[run_id]=get_metrics_by_hand(model, [normal_test_dataloader])

{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_old_run_id': '7zi9x9w8', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 128, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
qgkxyze7
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8


100%|██████████| 559/559 [00:18<00:00, 30.40it/s]


{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_old_run_id': '7zi9x9w8', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 128, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
r1xr7g4v
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8


100%|██████████| 559/559 [00:17<00:00, 31.16it/s]


{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_old_run_id': '7zi9x9w8', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 32, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
itxri54t
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8


...

100%|██████████| 559/559 [00:17<00:00, 31.24it/s]


{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_old_run_id': '7zi9x9w8', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 2, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
xm3z645m
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8


100%|██████████| 559/559 [00:17<00:00, 31.23it/s]


{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_old_run_id': '7zi9x9w8', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 2, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0}
zf87zj40
Distnace Augmentation: True Clusters: True Categories: False Noise std:  0.01 base model 7zi9x9w8


100%|██████████| 559/559 [00:17<00:00, 31.30it/s]


In [28]:
manual_res_df=pd.DataFrame(results).T.reset_index()
manual_res_df[['DataLoader_id','hit','mrr']]=pd.DataFrame(manual_res_df[0].to_list(), columns=['DataLoader_id','hit','mrr'])
manual_res_df.rename(columns={'index':'run_id'}, inplace=True)
manual_res_df.drop(columns=0, inplace=True)

In [27]:
fullres_df

,test_loss,test_hit,test_mrr,run_id,augment_nogmm,DataLoader_id,hit,mrr
0,4.364502,69.918480,30.679390,qgkxyze7,128,0,0.699184,0.306791
1,4.378706,70.149246,30.866415,r1xr7g4v,128,0,0.701492,0.308662
2,4.376307,70.070602,30.739613,itxri54t,32,0,0.700705,0.307394
3,4.375030,70.036583,30.747183,7vvqd0ib,32,0,0.700365,0.307469
4,4.374988,70.043739,30.704414,jefoas5f,8,0,0.700437,0.307042
5,4.379937,70.111748,30.824820,ai6ytfw2,8,0,0.701116,0.308246
6,4.372421,69.931038,30.606474,xm3z645m,2,0,0.699309,0.306063
7,4.360525,70.224342,30.744629,zf87zj40,2,0,0.702243,0.307444


In [26]:
res_df=res_df.merge(manual_res_df,  on='run_id')
res_df.drop(columns='run_id').groupby('augment_nogmm').mean()

,test_loss,test_hit,test_mrr,DataLoader_id,hit,mrr
augment_nogmm,,,,,,
2,4.366473,70.077690,30.675551,0.0,0.700776,0.306754
8,4.377462,70.077744,30.764617,0.0,0.700776,0.307644
32,4.375669,70.053593,30.743398,0.0,0.700535,0.307431
128,4.371604,70.033863,30.772902,0.0,0.700338,0.307727


## final comparison 

In [15]:
runs="""jcohtdpz
c7lixdzn
9r9e7dsx
lx4jtu4g
au07imlq
u337t04h
k7eoaxze
ln32zyo6
ud3mdfu3
zw2j8lcv
k598voxs
r5p9ms90
qhcln02h
1ttxe7ox
cqw7xz4i
gd66gitx
alh21te8
da4ptrb0
khflo2wo
uyyjj6l8
554tdh98
eoart48b
had1602b
0d2jc7tz
8r761av3
svgb60go
4627kuot
ye95z6m4""".split('\n')


In [27]:
runs=['ahsuiwog', 'e8j91dlm', 'sn0yz16r', 'zhy3fnyf', 'c31yywda']

In [28]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue

    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
ahsuiwog
Testing DataLoader 0: 100%|██████████| 559/559 [00:17<00:00, 32.47it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
    

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


e8j91dlm
Testing DataLoader 0: 100%|██████████| 559/559 [00:16<00:00, 32.94it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.12599182128906
        test_loss            4.360414505004883
        test_mrr            30.701974868774414
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'aug

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


sn0yz16r
Testing DataLoader 0: 100%|██████████| 559/559 [00:18<00:00, 30.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             69.80223083496094
        test_loss            4.375327110290527
        test_mrr            30.693470001220703
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'aug

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


zhy3fnyf
Testing DataLoader 0: 100%|██████████| 559/559 [00:18<00:00, 29.83it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             69.92022705078125
        test_loss            4.361334800720215
        test_mrr            30.723155975341797
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'aug

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


c31yywda
Testing DataLoader 0: 100%|██████████| 559/559 [00:18<00:00, 30.02it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.02037048339844
        test_loss            4.36799430847168
        test_mrr             30.69025993347168
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [29]:
result_df=pd.DataFrame(res_df)

In [30]:
# noise only
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_alg']].groupby(['augment_matrix','augment_alg']).mean()

,,test_loss,test_hit,test_mrr
augment_matrix,augment_alg,,,
False,gmm,4.365023,69.97034,30.675754


In [19]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_alg']].groupby(['augment_matrix','augment_alg']).mean()

test_loss   test_hit   test_mrr
augment_matrix augment_alg                                 
False          gmm           4.363266  69.939949  30.681960
True           categories    4.367515  70.021247  30.752016
               gmm           4.366891  70.019025  30.759678
               kmeans        4.365834  70.035826  30.689075
               raw           4.363883  70.090128  30.691610

# yoochoose 1/4

In [21]:
runs="""3j5nkw4m
fxrwajhe
vmbjvdhs
6vpf6swz
hbuhmnxw
za9sjn76
bchj1eho""".split('\n')


In [22]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue

    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'yoochoose1_4', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'fxrwajhe', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'categories', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
3j5nkw4m
Testing DataLoader 0: 100%|██████████| 559/559 [01:02<00:00,  8.96it/s]
──────────────────────────────────────────────────────────────────────────────────────────────────────────────────

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


fxrwajhe
Testing DataLoader 0: 100%|██████████| 559/559 [00:51<00:00, 10.84it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             71.28878021240234
        test_loss            4.212982654571533
        test_mrr             31.80596160888672
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_4', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augme

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

hbuhmnxw
Testing DataLoader 0: 100%|██████████| 559/559 [01:01<00:00,  9.02it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             71.1151123046875
        test_loss            4.214593410491943
        test_mrr             31.68343162536621
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_4', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augmen

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


za9sjn76
Testing DataLoader 0: 100%|██████████| 559/559 [00:51<00:00, 10.78it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             71.3692855834961
        test_loss            4.21355676651001
        test_mrr             31.82353973388672
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_4', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augmen

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


bchj1eho
Testing DataLoader 0: 100%|██████████| 559/559 [01:01<00:00,  9.04it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             71.40144348144531
        test_loss            4.213388919830322
        test_mrr            31.830402374267578
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [23]:
result_df=pd.DataFrame(res_df)

In [26]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_alg']]

,test_loss,test_hit,test_mrr,augment_matrix,augment_alg
0,4.211008,71.193970,31.710821,True,categories
1,4.212983,71.288780,31.805962,False,gmm
2,4.210006,71.295898,31.704958,True,raw
3,4.218857,71.142014,31.588299,True,kmeans
4,4.214593,71.115112,31.683432,True,gmm
5,4.213557,71.369286,31.823540,True,categories
6,4.213389,71.401443,31.830402,False,gmm


In [24]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_alg']].groupby(['augment_matrix','augment_alg']).mean()

test_loss   test_hit   test_mrr
augment_matrix augment_alg                                 
False          gmm           4.213186  71.345112  31.818182
True           categories    4.212282  71.281628  31.767180
               gmm           4.214593  71.115112  31.683432
               kmeans        4.218857  71.142014  31.588299
               raw           4.210006  71.295898  31.704958

# diginetica

## baseline models

In [4]:
run_id='qm2ur7o3'

model,opt=load_model(run_id)
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': None, 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}


In [5]:
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )

data masking start


100%|██████████| 64/64 [00:00<00:00, 222.72it/s]

done masking


In [6]:
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
runs=[ ## default hparams, as in paper
    'run-20240916_203347-izcd5fci',
    'run-20240916_200756-q3fjvapa',
]

In [13]:
runs=[
    ## my hparams
    'qm2ur7o3',
    '3abge2uq',
    '4dm99qnd',
    'jxgwsuta',
]

In [16]:
res_df=[]
for run_id in runs:
    model,opt=load_model(run_id)
    print('Metrics on normal Adjacency matrix')
    print(run_id)
    metrics=trainer.test(model, normal_test_dataloader)[0]
    metrics['run_id']=run_id
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': None, 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 1.0, 'augment_alg': 'gmm'}
Metrics on normal Adjacency matrix
qm2ur7o3
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 34.19it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
───────────────────────────────

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Metrics on normal Adjacency matrix
3abge2uq
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 33.84it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            50.733821868896484
        test_loss            5.508734226226807
        test_mrr             17.61170768737793
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': True, 'augment_matrix': False, 'augment_clusters': False

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Metrics on normal Adjacency matrix
4dm99qnd
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 33.91it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            50.689205169677734
        test_loss            5.500315189361572
        test_mrr            17.721900939941406
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': True, 'augment_matrix': False, 'augment_clusters': False

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Metrics on normal Adjacency matrix
jxgwsuta
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 35.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.92146301269531
        test_loss            5.515181541442871
        test_mrr            17.742328643798828
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [19]:
pd.DataFrame(res_df)[['test_loss','test_hit','test_mrr']].mean()

test_loss     5.508268
test_hit     50.797606
test_mrr     17.718086
dtype: float64

## only noise

In [10]:
runs=['s4uxxqad', 'rbwxpsy6', 'ba46mnkl', 'h2a8ujnt']

In [11]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)
res_df=pd.DataFrame(res_df)
res_df.groupby('augment_matrix')[['test_loss', 'test_hit', 'test_mrr',]].mean()

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


s4uxxqad
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 29.83it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.72368621826172
        test_loss            5.51453161239624
        test_mrr             17.66588020324707
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_c

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


rbwxpsy6
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 29.97it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            50.624542236328125
        test_loss           5.5087385177612305
        test_mrr            17.690868377685547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ba46mnkl
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 30.15it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.58184814453125
        test_loss            5.524074554443359
        test_mrr             17.53502655029297
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


h2a8ujnt
Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 30.56it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.28316879272461
        test_loss            5.53749418258667
        test_mrr            17.574359893798828
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


,test_loss,test_hit,test_mrr
augment_matrix,,,
False,5.52121,50.553311,17.616534


## augmented models

In [21]:
runs=[ 
    '9kf534bm',
    'nbhakjb7',
    'fmm07us9',
    'wtqp9kti',
    'dlnjkkym',
    'rrqcjbjh',
    '4kxamho0',
    'cx8cnx7m',    
    'seb1ybp1',    
    'bgqjpxh8',    
    'r8d4fcqv',    
    'ey3bjtz5',    
    '8npur3kf',  
    '21wtiveg',
    'run-20240916_191647-vyrrq40z',
    'run-20240916_185628-j8kqiv89',
]

In [12]:
runs=[
    'run-20240916_191647-vyrrq40z',
    'run-20240916_185628-j8kqiv89',
]

In [21]:
### 4 times per each # gmms
runs=['yp1n5fb3', 'naj1qrsy', 'pl1wvihn', 's7r181i0', 'fp4lprkj',
       'yrq35hmp', 'pxgponas', '31hc2s4t', 'v94f9jmw', '9wa54kna',
       'xbpg8ncx', 'kltkp2te', 'z7mdp78e', 'o9t9gt9v', 'qdxqwnrw',
       '0zl3ui7x', '7jkmaij6', 'ud4ap40l', '5k1ezsql', 'zkgt4s6x',
       'egqyw7yg', '6ixyzjk9', 'ub1h5qr3', '8iea3bxp', '8f9v8z85',
       'f80nrp4r', 'b60x4mtc', 'm8n6p80h', '26kyds0s', '628jlwv2',
       'dwgw5a3v', '18j6ce86', 'b5pfaja0', 'rwhslx2m', 'gt0rzzmz',
       'xscwi8xw', 'kkpuxgtn']+['c7pzwt2z', '17uejl8y', '57zrr3vh', 'op22qkq4']+['uxt9w9tv', '1qly30ga', 'y7yqfh84', '6i71w436']

In [31]:
runs=[
    'run-20241002_162623-kkpuxgtn',
    'run-20241002_160557-9ueod1xy',
]

In [22]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 128, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm'}
yp1n5fb3
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 30.13it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


naj1qrsy
Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 30.52it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.04846954345703
        test_loss            5.495818138122559
        test_mrr            18.010406494140625
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

1qly30ga
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 33.38it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.17955017089844
        test_loss            5.481403350830078
        test_mrr            17.958845138549805
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


y7yqfh84
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 34.01it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            51.176719665527344
        test_loss            5.494831562042236
        test_mrr            18.133237838745117
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


6i71w436
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 33.37it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.01042175292969
        test_loss            5.497133255004883
        test_mrr             18.04057502746582
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [23]:
res_df=pd.DataFrame(res_df)

In [12]:
res_df.drop(columns='run_id').groupby(['augment_nogmm','augment_old_run_id'])[['test_loss', 'test_hit', 'test_mrr',]].mean()

test_loss   test_hit   test_mrr
augment_nogmm augment_old_run_id                                 
4             3abge2uq             5.506084  50.862637  17.898970
              4dm99qnd             5.500349  51.075733  18.032616
              jxgwsuta             5.497977  51.057528  18.015642
              qm2ur7o3             5.508249  51.285545  18.009550
8             3abge2uq             5.499101  51.031765  17.970356
              4dm99qnd             5.493108  51.242483  18.063574
              jxgwsuta             5.493937  51.102600  17.998289
              qm2ur7o3             5.510464  51.008324  17.987402
16            3abge2uq             5.495231  51.219296  18.068703
              4dm99qnd             5.498574  50.971096  17.912088
              jxgwsuta             5.501109  50.919512  17.997243
              qm2ur7o3             5.502841  51.032673  17.906479
32            3abge2uq             5.493803  51.260914  18.043545
              4dm99qnd             5.504844  50.929081  17.990470
              jxgwsuta             5.486067  51.158146  18.046724
              qm2ur7o3             5.544961  50.620209  17.768856
64            3abge2uq             5.508237  51.093880  18.020321
              4dm99qnd             5.497005  51.140451  17.973605
              jxgwsuta             5.501431  50.965036  17.981283
              qm2ur7o3             5.499537  51.079384  18.044710
128           3abge2uq             5.496712  50.986694  18.062107
              4dm99qnd             5.495254  51.086493  18.007488
              jxgwsuta             5.493837  51.103874  18.031009
              qm2ur7o3             5.489111  51.016705  17.980143

In [24]:
res_df.drop(columns='run_id').groupby(['augment_alg','augment_nogmm'])[['test_loss', 'test_hit', 'test_mrr',]].mean()

test_loss   test_hit   test_mrr
augment_alg augment_nogmm                                 
categories  16              5.492075  51.081635  18.021056
gmm         4               5.501831  51.069117  18.000839
            8               5.497276  51.121709  18.013581
            16              5.499573  51.005531  17.965641
            32              5.509364  50.953684  17.950806
            64              5.500774  51.064040  17.995801
            128             5.494001  51.064022  18.019874
kmeans      8               5.492455  51.122759  18.057717

In [27]:
res_df.loc[res_df.augment_nogmm==8][['test_loss', 'test_hit', 'test_mrr','augment_alg', 'augment_old_run_id']]

,test_loss,test_hit,test_mrr,augment_alg,augment_old_run_id
4,5.491195,51.053566,17.951180,gmm,jxgwsuta
10,5.489055,51.150162,18.036987,gmm,4dm99qnd
16,5.496680,51.151634,18.045399,gmm,jxgwsuta
22,5.497161,51.334805,18.090160,gmm,4dm99qnd
28,5.499101,51.031765,17.970356,gmm,3abge2uq
34,5.510464,51.008324,17.987402,gmm,qm2ur7o3
41,5.496451,51.124344,18.098211,kmeans,qm2ur7o3
42,5.481403,51.179550,17.958845,kmeans,3abge2uq
43,5.494832,51.176720,18.133238,kmeans,4dm99qnd
44,5.497133,51.010422,18.040575,kmeans,jxgwsuta


In [13]:
display(res_df)

,test_loss,test_hit,test_mrr,run_id,dataset,batchSize,hiddenSize,epoch,lr,lr_dc,...,weight_init,augment_categories,augment_nogmm,augment_p,augment_mean,augment_std,augment_gmm_init,augment_prenormalize_distances,augment_noise_p,augment_alg
0,5.498109,51.115963,18.006958,yp1n5fb3,diginetica,100,100,30,0.001,0.1,...,normal,False,128,0.5,0.01,0.01,k-means++,False,0.5,gmm
1,5.495818,51.048470,18.010406,naj1qrsy,diginetica,100,100,30,0.001,0.1,...,normal,False,64,0.5,0.01,0.01,k-means++,False,0.5,gmm
2,5.480463,51.300774,18.113754,pl1wvihn,diginetica,100,100,30,0.001,0.1,...,normal,False,32,0.5,0.01,0.01,k-means++,False,0.5,gmm
3,5.503563,50.868919,18.040007,s7r181i0,diginetica,100,100,30,0.001,0.1,...,normal,False,16,0.5,0.01,0.01,k-means++,False,0.5,gmm
4,5.491195,51.053566,17.951180,fp4lprkj,diginetica,100,100,30,0.001,0.1,...,normal,False,8,0.5,0.01,0.01,k-means++,False,0.5,gmm
5,5.505888,50.985050,17.990795,yrq35hmp,diginetica,100,100,30,0.001,0.1,...,normal,False,4,0.5,0.01,0.01,k-means++,False,0.5,gmm
6,5.500285,50.961330,18.010942,pxgponas,diginetica,100,100,30,0.001,0.1,...,normal,False,128,0.5,0.01,0.01,k-means++,False,0.5,gmm
7,5.496689,51.138214,17.988705,31hc2s4t,diginetica,100,100,30,0.001,0.1,...,normal,False,64,0.5,0.01,0.01,k-means++,False,0.5,gmm
8,5.506217,50.930130,18.019054,v94f9jmw,diginetica,100,100,30,0.001,0.1,...,normal,False,32,0.5,0.01,0.01,k-means++,False,0.5,gmm
9,5.494094,50.932056,17.878902,9wa54kna,diginetica,100,100,30,0.001,0.1,...,normal,False,16,0.5,0.01,0.01,k-means++,False,0.5,gmm


In [14]:
results={}
for run_id in runs:
  try:
    model,opt=load_model(run_id)
  except IndexError:
      continue    #dataset=get_dataset(opt)
  # am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
  #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
  #                   )
  print(run_id)
  print('Distnace Augmentation:', opt.augment_matrix,
      'Clusters:', opt.augment_clusters,
        'Categories:', opt.augment_categories,
        'Noise std: ', opt.augment_std,
        'base model', opt.augment_old_run_id,
        )
  metrics=get_metrics_by_hand(model, [normal_test_dataloader])
  results[run_id]=metrics
  print(metrics)

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 128, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm'}


yp1n5fb3
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model jxgwsuta


100%|██████████| 609/609 [00:21<00:00, 27.72it/s]


[(0, 0.511107824772421, 0.18002728628510217)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 64, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm'}
naj1qrsy
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model jxgwsuta


100%|██████████| 609/609 [00:21<00:00, 27.81it/s]


...

[(0, 0.5100069013112492, 0.17980339839575524)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 4, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm'}
xscwi8xw
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model qm2ur7o3


100%|██████████| 609/609 [00:22<00:00, 27.57it/s]


[(0, 0.5128167208912551, 0.18006235085203398)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 32, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm'}
kkpuxgtn
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model qm2ur7o3


100%|██████████| 609/609 [00:21<00:00, 27.74it/s]


[(0, 0.501758191199185, 0.17445833306371644)]


In [16]:
manual_res_df=pd.DataFrame(results).T.reset_index()
manual_res_df[['DataLoader_id','hit','mrr']]=pd.DataFrame(manual_res_df[0].to_list(), columns=['DataLoader_id','hit','mrr'])
manual_res_df.rename(columns={'index':'run_id'}, inplace=True)
manual_res_df.drop(columns=0, inplace=True)

In [20]:
res_df=res_df.merge(manual_res_df,  on='run_id')

,run_id,DataLoader_id,hit,mrr
0,yp1n5fb3,0,0.511108,0.180027
1,naj1qrsy,0,0.510385,0.180053
2,pl1wvihn,0,0.512981,0.181123
3,s7r181i0,0,0.508660,0.180362
4,fp4lprkj,0,0.510483,0.179483
5,yrq35hmp,0,0.509810,0.179867
6,pxgponas,0,0.509596,0.180071
7,31hc2s4t,0,0.511354,0.179862
8,v94f9jmw,0,0.509284,0.180181
9,9wa54kna,0,0.509267,0.178781


In [22]:
res_df.head()

,test_loss,test_hit,test_mrr,run_id,dataset,batchSize,hiddenSize,epoch,lr,lr_dc,...,augment_gmm_init,augment_prenormalize_distances,augment_noise_p,augment_alg,DataLoader_id_x,hit_x,mrr_x,DataLoader_id_y,hit_y,mrr_y
0,5.498109,51.115963,18.006958,yp1n5fb3,diginetica,100,100,30,0.001,0.1,...,k-means++,False,0.5,gmm,0,0.511108,0.180027,0,0.511108,0.180027
1,5.495818,51.048470,18.010406,naj1qrsy,diginetica,100,100,30,0.001,0.1,...,k-means++,False,0.5,gmm,0,0.510385,0.180053,0,0.510385,0.180053
2,5.480463,51.300774,18.113754,pl1wvihn,diginetica,100,100,30,0.001,0.1,...,k-means++,False,0.5,gmm,0,0.512981,0.181123,0,0.512981,0.181123
3,5.503563,50.868919,18.040007,s7r181i0,diginetica,100,100,30,0.001,0.1,...,k-means++,False,0.5,gmm,0,0.508660,0.180362,0,0.508660,0.180362
4,5.491195,51.053566,17.951180,fp4lprkj,diginetica,100,100,30,0.001,0.1,...,k-means++,False,0.5,gmm,0,0.510483,0.179483,0,0.510483,0.179483


In [26]:
res_df.drop(columns='run_id').groupby('augment_nogmm')[['test_loss', 'test_hit', 'hit_x','test_mrr','mrr_x']].mean()

,test_loss,test_hit,hit_x,test_mrr,mrr_x
augment_nogmm,,,,,
4,5.501831,51.069117,51.064500,18.000839,17.996884
8,5.497276,51.121709,51.116534,18.013581,18.008950
16,5.499573,51.005531,51.002059,17.965641,17.962356
32,5.509364,50.953684,50.949752,17.950806,17.947840
64,5.500774,51.064040,51.059023,17.995801,17.992641
128,5.494001,51.064022,51.060392,18.019874,18.015939


## categories runs for comparison

In [2]:
runs=['c7pzwt2z', '17uejl8y', '57zrr3vh', 'op22qkq4']

In [9]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'categories'}
c7pzwt2z


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 31.41it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            50.927127838134766
        test_loss            5.484939098358154
        test_mrr            17.949098587036133
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'augment_cl

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


17uejl8y
Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 31.65it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.25361251831055
        test_loss            5.490333557128906
        test_mrr            18.101985931396484
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


57zrr3vh
Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 31.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit               51.0791015625
        test_loss            5.497370719909668
        test_mrr             18.04535484313965
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'aug

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


op22qkq4
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 32.48it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.06669998168945
        test_loss            5.495656490325928
        test_mrr             17.98778533935547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [10]:
res_df=pd.DataFrame(res_df)

In [11]:
res_df.drop(columns='run_id').groupby(['augment_nogmm'])[['test_loss', 'test_hit', 'test_mrr',]].mean()

,test_loss,test_hit,test_mrr
augment_nogmm,,,
16,5.492075,51.081635,18.021056


## i2i distance

In [18]:
runs=['yfd2pi3n', 'ss45a3b3', 'nwgcf1hh', '8llxhkxm']

In [19]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
yfd2pi3n
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 29.01it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ss45a3b3
Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 32.04it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.25015640258789
        test_loss            5.510570526123047
        test_mrr            18.078449249267578
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


nwgcf1hh
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 34.02it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            51.276145935058594
        test_loss            5.499122142791748
        test_mrr             18.04974365234375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


8llxhkxm
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 34.17it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.24868392944336
        test_loss            5.488428115844727
        test_mrr            18.089075088500977
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [20]:
res_df=pd.DataFrame(res_df)

In [21]:
res_df.drop(columns='run_id').groupby(['augment_nogmm'])[['test_loss', 'test_hit', 'test_mrr',]].mean()

,test_loss,test_hit,test_mrr
augment_nogmm,,,
16,5.498733,51.179676,18.054538


In [22]:
results={}
for run_id in runs:
  try:
    model,opt=load_model(run_id)
  except IndexError:
      continue    #dataset=get_dataset(opt)
  # am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
  #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
  #                   )
  print(run_id)
  print('Distnace Augmentation:', opt.augment_matrix,
      'Clusters:', opt.augment_clusters,
        'Categories:', opt.augment_categories,
        'Noise std: ', opt.augment_std,
        'base model', opt.augment_old_run_id,
        )
  metrics=get_metrics_by_hand(model, [normal_test_dataloader])
  results[run_id]=metrics
  print(metrics)
manual_res_df=pd.DataFrame(results).T.reset_index()
manual_res_df[['DataLoader_id','hit','mrr']]=pd.DataFrame(manual_res_df[0].to_list(), columns=['DataLoader_id','hit','mrr'])
manual_res_df.rename(columns={'index':'run_id'}, inplace=True)
manual_res_df.drop(columns=0, inplace=True)
res_df=res_df.merge(manual_res_df,  on='run_id')

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
yfd2pi3n
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model qm2ur7o3


100%|██████████| 609/609 [00:21<00:00, 28.71it/s]


[(0, 50.943179204048775, 17.997264953220483)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
ss45a3b3
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model 3abge2uq


100%|██████████| 609/609 [00:22<00:00, 27.27it/s]


[(0, 51.24387919418976, 18.071916962386005)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
nwgcf1hh
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model 4dm99qnd


100%|██████████| 609/609 [00:24<00:00, 24.77it/s]


[(0, 51.27345624240034, 18.04877398974439)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
8llxhkxm
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model jxgwsuta


100%|██████████| 609/609 [00:23<00:00, 25.81it/s]


[(0, 51.247165532879826, 18.086323592674873)]


In [23]:
res_df.drop(columns='run_id').groupby('augment_nogmm')[['test_loss', 'test_hit', 'hit','test_mrr','mrr']].mean()

,test_loss,test_hit,hit,test_mrr,mrr
augment_nogmm,,,,,
16,5.498733,51.179676,51.17692,18.054538,18.05107


## i2i no normalization aug_p=1

In [8]:
runs=['s2qhugqc', '1jvw1mc3', 'gpx6kyhh', 'vx4bvbxf']

In [9]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)
res_df=pd.DataFrame(res_df)
res_df.groupby('augment_matrix')[['test_loss', 'test_hit', 'test_mrr',]].mean()

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
s2qhugqc


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 609/609 [00:21<00:00, 28.10it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.3122673034668
        test_loss            5.534583568572998
        test_mrr            17.871023178100586
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'augment_cli

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1jvw1mc3
Testing DataLoader 0: 100%|██████████| 609/609 [00:21<00:00, 28.91it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.04954528808594
        test_loss            5.510757923126221
        test_mrr            18.006027221679688
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


gpx6kyhh
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 29.32it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.89785385131836
        test_loss           5.5169572830200195
        test_mrr            18.005224227905273
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


vx4bvbxf
Testing DataLoader 0: 100%|██████████| 609/609 [00:20<00:00, 29.53it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.84100341796875
        test_loss            5.507208347320557
        test_mrr            17.996774673461914
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


,test_loss,test_hit,test_mrr
augment_matrix,,,
True,5.517377,50.775167,17.969762


## i2i no normalization

In [7]:
runs=['o6eky4nz', '0todhcp5', 'czhmlmnc', 'cc0e54iw']

In [8]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)
res_df=pd.DataFrame(res_df)
res_df.groupby('augment_matrix')[['test_loss', 'test_hit', 'test_mrr',]].mean()

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
o6eky4nz


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 31.22it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.89994812011719
        test_loss            5.515530586242676
        test_mrr            17.970956802368164
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'augment_cl

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


0todhcp5
Testing DataLoader 0: 100%|██████████| 609/609 [00:21<00:00, 27.81it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.02610397338867
        test_loss            5.518783092498779
        test_mrr             17.98603630065918
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 609/609 [00:21<00:00, 28.68it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.85396957397461
        test_loss            5.511118412017822
        test_mrr            18.009906768798828
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'augment_cl

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


cc0e54iw
Testing DataLoader 0: 100%|██████████| 609/609 [00:19<00:00, 30.61it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            51.077457427978516
        test_loss            5.496988296508789
        test_mrr            18.073209762573242
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


,test_loss,test_hit,test_mrr
augment_matrix,,,
True,5.510605,50.96437,18.010027


## kmeans augmented runs

In [12]:
runs=['uxt9w9tv', '1qly30ga', 'y7yqfh84', '6i71w436']

In [13]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'kmeans'}


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


uxt9w9tv
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 32.41it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.12434387207031
        test_loss            5.496450901031494
        test_mrr             18.09821128845215
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1qly30ga
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 32.27it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.17955017089844
        test_loss            5.481403350830078
        test_mrr            17.958845138549805
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


y7yqfh84
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 32.23it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            51.176719665527344
        test_loss            5.494831562042236
        test_mrr            18.133237838745117
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


6i71w436
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 32.27it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.01042175292969
        test_loss            5.497133255004883
        test_mrr             18.04057502746582
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [14]:
res_df=pd.DataFrame(res_df)

In [15]:
res_df.drop(columns='run_id').groupby(['augment_nogmm'])[['test_loss', 'test_hit', 'test_mrr',]].mean()

,test_loss,test_hit,test_mrr
augment_nogmm,,,
8,5.492455,51.122759,18.057717


In [18]:
results={}
for run_id in runs:
  try:
    model,opt=load_model(run_id)
  except IndexError:
      continue    #dataset=get_dataset(opt)
  # am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
  #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
  #                   )
  print(run_id)
  print('Distnace Augmentation:', opt.augment_matrix,
      'Clusters:', opt.augment_clusters,
        'Categories:', opt.augment_categories,
        'Noise std: ', opt.augment_std,
        'base model', opt.augment_old_run_id,
        )
  metrics=get_metrics_by_hand(model, [normal_test_dataloader])
  results[run_id]=metrics
  print(metrics)
manual_res_df=pd.DataFrame(results).T.reset_index()
manual_res_df[['DataLoader_id','hit','mrr']]=pd.DataFrame(manual_res_df[0].to_list(), columns=['DataLoader_id','hit','mrr'])
manual_res_df.rename(columns={'index':'run_id'}, inplace=True)
manual_res_df.drop(columns=0, inplace=True)
res_df=res_df.merge(manual_res_df,  on='run_id')

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'kmeans'}
uxt9w9tv
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model qm2ur7o3


100%|██████████| 609/609 [00:20<00:00, 29.95it/s]


[(0, 51.123927832002366, 18.092726716649587)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'kmeans'}
1qly30ga
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model 3abge2uq


100%|██████████| 609/609 [00:20<00:00, 30.41it/s]


[(0, 51.173222912353346, 17.95476602952377)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'kmeans'}
y7yqfh84
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model 4dm99qnd


100%|██████████| 609/609 [00:19<00:00, 30.50it/s]


[(0, 51.17157974300831, 18.128069937516443)]
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'kmeans'}
6i71w436
Distnace Augmentation: True Clusters: False Categories: False Noise std:  0.01 base model jxgwsuta


100%|██████████| 609/609 [00:20<00:00, 30.25it/s]

[(0, 51.00397646981498, 18.036856236887587)]


In [20]:
res_df.drop(columns='run_id').groupby('augment_nogmm')[['test_loss', 'test_hit', 'hit','test_mrr','mrr']].mean()

,test_loss,test_hit,hit,test_mrr,mrr
augment_nogmm,,,,,
8,5.492455,51.122759,51.118177,18.057717,18.053105


## hidden size = 16

In [4]:
run_id='run-20240930_183501-kmnrqi6i'

aug_run_ids=[
    'run-20241002_140249-xros8ub6',
    'run-20241002_131153-wijus07w',
    'run-20241002_124004-pkv65nix',
    'run-20241002_121528-v3ukvj2b',
]

In [5]:
model,opt=load_model(run_id)
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 16, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm'}
data masking start


100%|██████████| 64/64 [00:00<00:00, 253.36it/s]

done masking


In [6]:
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [7]:
for test_run_id in [run_id]+aug_run_ids:
    model,opt=load_model(test_run_id)

    print(test_run_id, opt.augment_matrix, opt.augment_alg)
    trainer.test(model, normal_test_dataloader)

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 16, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm'}
run-20240930_183501-kmnrqi6i False gmm


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 34.57it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             45.66830825805664
        test_loss            6.029724597930908
        test_mrr            14.656782150268555
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 16, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'kmnrqi6i', 'augment_cli

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


run-20241002_140249-xros8ub6 True kmeans
Testing DataLoader 0: 100%|██████████| 609/609 [00:16<00:00, 37.46it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            44.556819915771484
        test_loss            6.107140064239502
        test_mrr            13.926748275756836
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 16, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augme

...

run-20241002_131153-wijus07w True gmm
Testing DataLoader 0: 100%|██████████| 609/609 [00:16<00:00, 36.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             45.28990173339844
        test_loss            6.080326080322266
        test_mrr            14.271702766418457
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 16, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


run-20241002_124004-pkv65nix True gmm
Testing DataLoader 0: 100%|██████████| 609/609 [00:17<00:00, 35.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             42.88568115234375
        test_loss            6.296449661254883
        test_mrr            13.708586692810059
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 16, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': True, 'augment_

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


run-20241002_121528-v3ukvj2b True gmm
Testing DataLoader 0: 100%|██████████| 609/609 [00:18<00:00, 33.49it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            42.846553802490234
        test_loss            6.29763650894165
        test_mrr            13.632750511169434
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


## augment + GM layer

In [7]:
runs=['v9h49456', 'c7ihc3ll', 'jadnjsru', 'pbreuu0o']
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model_gm(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)
res_df=pd.DataFrame(res_df)
res_df.groupby('augment_matrix')[['test_loss', 'test_hit', 'test_mrr',]].mean()

{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
v9h49456


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 609/609 [00:42<00:00, 14.20it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            50.240020751953125
        test_loss            5.570756435394287
        test_mrr            17.456981658935547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '3abge2uq', 'augment_c

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


c7ihc3ll
Testing DataLoader 0: 100%|██████████| 609/609 [00:40<00:00, 14.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.01477813720703
        test_loss            5.594357967376709
        test_mrr            17.473312377929688
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', '

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


jadnjsru
Testing DataLoader 0: 100%|██████████| 609/609 [00:42<00:00, 14.24it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            49.979393005371094
        test_loss            5.582583904266357
        test_mrr            17.552082061767578
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 3, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', '

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


pbreuu0o
Testing DataLoader 0: 100%|██████████| 609/609 [00:42<00:00, 14.50it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             49.90504455566406
        test_loss            5.582030773162842
        test_mrr             17.40863037109375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


,test_loss,test_hit,test_mrr
augment_matrix,,,
False,5.582432,50.034809,17.472752


In [11]:
runs=['zq77vmnb','fpu7ucbf']+['12bdl9rw', 'wsix33ma', '1lh9waxz', 'nqk6cfps']+['n02c6569', 'q9w5okna', 'ltt7vsmv', '7o8vwuxv']

In [14]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model_gm(run_id)
    except IndexError:
       continue
    #dataset=get_dataset(opt)
    #am_test_dataloader=DataLoader(dataset,    num_workers=os.cpu_count(),  
    #                       sampler=SRGNN_sampler(dataset, opt.batchSize, shuffle=False, drop_last=False)
    #                     )
    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'raw', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
zq77vmnb
Testing DataLoader 0: 100%|██████████| 609/609 [00:38<00:00, 15.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
─────────────

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


fpu7ucbf
Testing DataLoader 0: 100%|██████████| 609/609 [00:38<00:00, 15.92it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.45914840698242
        test_loss            5.489494800567627
        test_mrr             17.94986343383789
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-06, 'step': 2, 'patience': 4, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'qm2ur7o3', 'a

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


...

q9w5okna
Testing DataLoader 0: 100%|██████████| 609/609 [00:38<00:00, 15.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.87010955810547
        test_loss            5.494775295257568
        test_mrr             17.70394515991211
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '4dm99qnd', '

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


ltt7vsmv
Testing DataLoader 0: 100%|██████████| 609/609 [00:37<00:00, 16.07it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.67634963989258
        test_loss           5.5109429359436035
        test_mrr            17.650726318359375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 100, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'l2': 1e-05, 'step': 2, 'patience': 5, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': 'jxgwsuta', '

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


7o8vwuxv
Testing DataLoader 0: 100%|██████████| 609/609 [00:38<00:00, 15.87it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.51378631591797
        test_loss            5.514265060424805
        test_mrr            17.573287963867188
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [15]:
res_df=pd.DataFrame(res_df)

In [16]:
res_df.groupby('augment_matrix')[['test_loss', 'test_hit', 'test_mrr',]].mean()

,test_loss,test_hit,test_mrr
augment_matrix,,,
False,5.511737,50.640451,17.638541
True,5.501566,51.168262,17.962647


In [17]:
res_df

,test_loss,test_hit,test_mrr,run_id,dataset,batchSize,hiddenSize,epoch,lr,lr_dc,...,augment_nogmm,augment_p,augment_mean,augment_std,augment_gmm_init,augment_prenormalize_distances,augment_noise_p,augment_alg,gmm_covariance_type,gmm_tol
0,5.507059,50.996998,17.954378,zq77vmnb,diginetica,100,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
1,5.489495,51.459148,17.949863,fpu7ucbf,diginetica,100,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
2,5.505618,51.093426,17.965767,12bdl9rw,diginetica,100,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
3,5.497803,51.173435,17.933750,wsix33ma,diginetica,100,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
4,5.504984,51.251801,18.049730,1lh9waxz,diginetica,100,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
5,5.504435,51.034767,17.922394,nqk6cfps,diginetica,100,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
6,5.526964,50.501560,17.626205,n02c6569,diginetica,100,100,30,0.001,0.1,...,8,1.0,0.01,0.00,k-means++,False,0.0,gmm,full,0.001
7,5.494775,50.870110,17.703945,q9w5okna,diginetica,100,100,30,0.001,0.1,...,8,1.0,0.01,0.00,k-means++,False,0.0,gmm,full,0.001
8,5.510943,50.676350,17.650726,ltt7vsmv,diginetica,100,100,30,0.001,0.1,...,8,1.0,0.01,0.00,k-means++,False,0.0,gmm,full,0.001
9,5.514265,50.513786,17.573288,7o8vwuxv,diginetica,100,100,30,0.001,0.1,...,8,1.0,0.01,0.00,k-means++,False,0.0,gmm,full,0.001


# TAGNN

## otto-recsys

In [6]:
runs = ['5f8cxkla', 'bqpaq7d1', 'onjpsmm2',
        '14oq6k8n',
  'uebl28zc',
  'tfeek0n2',
  '6u14u7kn',
  'gcvf1rcu',
  'ih8175d2',
  'l2w97z5v',
  '1uwq99zx',
  '4l1c4ls3']

In [7]:
run_id=runs[0]

model,opt=load_model(run_id)
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

{'dataset': 'otto-recsys', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
data masking start


100%|██████████| 64/64 [00:00<00:00, 300.93it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


done masking


/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue

    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

{'dataset': 'otto-recsys', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
5f8cxkla


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 980/980 [00:39<00:00, 24.60it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             34.12760925292969
        test_loss            7.512598037719727
        test_mrr            21.321727752685547
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_cluster

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


bqpaq7d1
Testing DataLoader 0: 100%|██████████| 980/980 [00:39<00:00, 24.96it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            34.087955474853516
        test_loss            7.474574565887451
        test_mrr            21.635311126708984
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augmen

...

Testing DataLoader 0: 100%|██████████| 980/980 [00:39<00:00, 24.90it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             35.04438400268555
        test_loss            7.434382915496826
        test_mrr            22.800670623779297
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1uwq99zx
Testing DataLoader 0: 100%|██████████| 980/980 [00:39<00:00, 24.98it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             34.95679473876953
        test_loss           7.4169511795043945
        test_mrr             22.72024917602539
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'otto-recsys', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 980/980 [00:39<00:00, 24.99it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit            34.947025299072266
        test_loss            7.417011260986328
        test_mrr            22.516698837280273
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [9]:
result_df=pd.DataFrame(res_df)

In [10]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_noise_p','augment_alg']].groupby(['augment_matrix', 'augment_noise_p', 'augment_alg']).mean()

test_loss   test_hit   test_mrr
augment_matrix augment_noise_p augment_alg                                 
False          0.0             gmm           7.502531  34.096923  21.433434
True           0.5             gmm           7.440699  35.023759  22.824743
                               kmeans        7.427819  35.079493  22.768881
                               raw           7.433808  35.037476  22.714960

## yoochoose 1/64

In [4]:
runs = [
 'r7bs25i7',
 '2ssf7s2q',
 'hucuxl8g',
 '1iwv7ul8',
 'sqvv043i',
 '6hkj647n'   
]

In [5]:
run_id=runs[0]

model,opt=load_model(run_id)
opt.batchSize=32
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

{'dataset': 'yoochoose1_64', 'batchSize': 48, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
data masking start


100%|██████████| 64/64 [00:00<00:00, 182.21it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


done masking


/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [6]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue

    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

{'dataset': 'yoochoose1_64', 'batchSize': 48, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augment_clusters': False, 'augment_old_run_id': '', 'augment_clip': 0, 'augment_normalize': False, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 16, 'augment_p': 1.0, 'augment_mean': 0.01, 'augment_std': 0.0, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.0, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
r7bs25i7


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1747/1747 [00:49<00:00, 35.17it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.45759582519531
        test_loss            4.322422027587891
        test_mrr            30.673080444335938
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 48, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clus

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


2ssf7s2q
Testing DataLoader 0: 100%|██████████| 1747/1747 [00:49<00:00, 35.58it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.5144271850586
        test_loss            4.308393478393555
        test_mrr            30.786468505859375
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 48, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augm

...

1iwv7ul8
Testing DataLoader 0: 100%|██████████| 1747/1747 [00:49<00:00, 35.63it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.78411865234375
        test_loss            4.299348831176758
        test_mrr             31.04060173034668
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 48, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'aug

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


sqvv043i
Testing DataLoader 0: 100%|██████████| 1747/1747 [00:48<00:00, 35.87it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.51580047607422
        test_loss           4.3084282875061035
        test_mrr             30.61737823486328
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'yoochoose1_64', 'batchSize': 48, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'au

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


6hkj647n
Testing DataLoader 0: 100%|██████████| 1747/1747 [00:47<00:00, 36.56it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             70.55377960205078
        test_loss            4.317729949951172
        test_mrr            30.842527389526367
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [7]:
result_df=pd.DataFrame(res_df)

In [8]:
result_df

,test_loss,test_hit,test_mrr,run_id,dataset,batchSize,hiddenSize,epoch,lr,lr_dc,...,augment_nogmm,augment_p,augment_mean,augment_std,augment_gmm_init,augment_prenormalize_distances,augment_noise_p,augment_alg,gmm_covariance_type,gmm_tol
0,4.322422,70.457596,30.673080,r7bs25i7,yoochoose1_64,48,100,30,0.001,0.1,...,16,1.0,0.01,0.00,k-means++,False,0.0,gmm,full,0.001
1,4.308393,70.514427,30.786469,2ssf7s2q,yoochoose1_64,48,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,raw,full,0.001
2,4.302582,70.721924,30.863203,hucuxl8g,yoochoose1_64,48,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,kmeans,full,0.001
3,4.299349,70.784119,31.040602,1iwv7ul8,yoochoose1_64,48,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,gmm,full,0.001
4,4.308428,70.515800,30.617378,sqvv043i,yoochoose1_64,48,100,30,0.001,0.1,...,8,0.5,0.01,0.01,k-means++,False,0.5,categories,full,0.001
5,4.317730,70.553780,30.842527,6hkj647n,yoochoose1_64,48,100,30,0.001,0.1,...,16,1.0,0.01,0.00,k-means++,False,0.0,gmm,full,0.001


In [9]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_noise_p','augment_alg']].groupby(['augment_matrix', 'augment_noise_p', 'augment_alg']).mean()

test_loss   test_hit   test_mrr
augment_matrix augment_noise_p augment_alg                                 
False          0.0             gmm           4.320076  70.505688  30.757804
True           0.5             categories    4.308428  70.515800  30.617378
                               gmm           4.299349  70.784119  31.040602
                               kmeans        4.302582  70.721924  30.863203
                               raw           4.308393  70.514427  30.786469

## Diginetica

In [4]:
runs = ['uasx1mwm', 'kig9t38y', 'i0o5vrfp', 'keifx1eb','w0dp5onn','paiznfvc']

In [5]:
run_id=runs[0]

model,opt=load_model(run_id)
opt.batchSize=32
test_data = pickle.load(open('../datasets/' + opt.dataset + '/test.txt', 'rb'))
normal_test_dataset=SRGNN_Map_Dataset(test_data, shuffle=False)

normal_test_dataloader=DataLoader(normal_test_dataset, 
                            num_workers=os.cpu_count(),  
                            sampler=SRGNN_sampler(normal_test_dataset, opt.batchSize, shuffle=False, drop_last=False)
                            )
trainer=Trainer(limit_test_batches=ceil(normal_test_dataset.length/opt.batchSize),
                limit_predict_batches=ceil(normal_test_dataset.length/opt.batchSize))

{'dataset': 'diginetica', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'paiznfvc', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
data masking start


100%|██████████| 64/64 [00:00<00:00, 256.67it/s]
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


done masking


/home/kacper/anaconda3/envs/gpu_torch/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [6]:
res_df=[]
for run_id in runs:
    try:
      model,opt=load_model(run_id)
    except IndexError:
       continue

    print(run_id)
    metrics=trainer.test(model, {#'augmented':am_test_dataloader, 
                                 'normal':normal_test_dataloader})[0 ]
    metrics['run_id']=run_id
    for k,v in opt.__dict__.items():
       metrics[k]=v
    res_df.append(metrics)

{'dataset': 'diginetica', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_clusters': False, 'augment_old_run_id': 'paiznfvc', 'augment_clip': 0, 'augment_normalize': True, 'augment_raw': False, 'weight_init': 'normal', 'augment_categories': False, 'augment_nogmm': 8, 'augment_p': 0.5, 'augment_mean': 0.01, 'augment_std': 0.01, 'augment_gmm_init': 'k-means++', 'augment_prenormalize_distances': False, 'augment_noise_p': 0.5, 'augment_alg': 'gmm', 'gmm_covariance_type': 'full', 'gmm_tol': 0.001}
uasx1mwm


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 1902/1902 [00:56<00:00, 33.41it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.13443374633789
        test_loss            5.567941665649414
        test_mrr            18.053564071655273
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment_cluster

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


kig9t38y
Testing DataLoader 0: 100%|██████████| 1902/1902 [00:56<00:00, 33.59it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.1052360534668
        test_loss            5.585332870483398
        test_mrr            18.082056045532227
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augment

...

keifx1eb
Testing DataLoader 0: 100%|██████████| 1902/1902 [00:57<00:00, 33.20it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.11104965209961
        test_loss           5.5915656089782715
        test_mrr            17.968429565429688
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-06, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': True, 'augmen

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


w0dp5onn
Testing DataLoader 0: 100%|██████████| 1902/1902 [00:58<00:00, 32.60it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             51.08552169799805
        test_loss            5.575977325439453
        test_mrr             18.00004005432129
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{'dataset': 'diginetica', 'batchSize': 64, 'hiddenSize': 100, 'epoch': 30, 'lr': 0.001, 'lr_dc': 0.1, 'lr_dc_step': 2, 'lr_scheduler': 'step', 'lr_milestones': ([2, 5, 8],), 'l2': 1e-05, 'step': 2, 'patience': 3, 'nonhybrid': False, 'validation': True, 'valid_portion': 0.1, 'pretrained_embedings': False, 'unfreeze_epoch': 0, 'gmm': [], 'augment_matrix': False, 'augme

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


paiznfvc
Testing DataLoader 0: 100%|██████████| 1902/1902 [01:01<00:00, 30.88it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_hit             50.34490203857422
        test_loss            5.572190284729004
        test_mrr             17.65895652770996
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [7]:
result_df=pd.DataFrame(res_df)

In [11]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'run_id','augment_matrix','augment_alg']]

,test_loss,test_hit,test_mrr,run_id,augment_matrix,augment_alg
0,5.567942,51.134434,18.053564,uasx1mwm,True,gmm
1,5.585333,51.105236,18.082056,kig9t38y,True,kmeans
2,5.572648,51.037495,18.170986,i0o5vrfp,True,raw
3,5.591566,51.111050,17.968430,keifx1eb,True,gmm
4,5.575977,51.085522,18.000040,w0dp5onn,True,categories
5,5.572190,50.344902,17.658957,paiznfvc,False,gmm


In [9]:
result_df[['test_loss', 'test_hit', 'test_mrr', 'augment_matrix','augment_noise_p','augment_alg']].groupby(['augment_matrix', 'augment_noise_p', 'augment_alg']).mean()

test_loss   test_hit   test_mrr
augment_matrix augment_noise_p augment_alg                                 
False          0.0             gmm           5.572190  50.344902  17.658957
True           0.5             categories    5.575977  51.085522  18.000040
                               gmm           5.579754  51.122742  18.010997
                               kmeans        5.585333  51.105236  18.082056
                               raw           5.572648  51.037495  18.170986